In [1]:
# All Necessary Libraries

import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_groq import ChatGroq
from typing import Annotated
from langchain.chat_models import init_chat_model
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import List, Literal
from typing_extensions import TypedDict
from langchain_core.documents import Document
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import END, StateGraph, START
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools import DuckDuckGoSearchRun

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Load the datasource
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs =[]
for url in urls:
    loader = WebBaseLoader(web_paths=[url])
    async for doc in loader.alazy_load():
        docs.append(doc)
print("The length of documents after loading",len(docs))

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  6.00it/s]

The length of documents after loading 3


In [3]:
# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
documents = text_splitter.split_documents(docs)
print("The length of documents after loading",len(documents))

The length of documents after loading 184


In [4]:
# Initate a embedding 
from langchain_huggingface import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="ibm-granite/granite-embedding-125m-english")
# Initate the vector database with the embedding model and pass the documents
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings_model,
    persist_directory="./chroma_db",  # Where to save data locally, remove if not necessary
)
vector_store.add_documents(documents=documents)

d:\Generative AI\Langchain\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['f70a6daf-4751-4904-9ae4-e21d4f5bc20d',
 '45ba1685-08a3-44d2-b535-e12d8bb4a6dd',
 '2713f8cf-da66-4a85-aead-e3534d819836',
 '2a4cdfc5-73ae-4672-8643-bc9cbddb7308',
 'b35847f2-cd6b-4651-b4ee-1e29d0010819',
 '71aaacfd-bd40-42e2-9d79-7bcc0b1f3a93',
 'd6e25106-8110-4c71-abed-7cca040b7c32',
 '7adf38e1-f688-444c-a2c6-2a5ccd366a13',
 'd62b8adf-97a3-4405-87e8-6ee56ab6a9f4',
 'ffed46d4-2dc3-4684-b37c-c08948807fa0',
 '47f1f306-62b1-4dc0-af7d-0b0001b50364',
 'fdad7ae3-4b1b-41ad-9dc3-7d19c315d27a',
 '5b209f62-f2d2-4d72-a96e-c3a735c9d8a6',
 '99d7a1f7-91f2-4917-849d-25500f1ccf22',
 '6f8bd8b0-4dec-400a-9d6b-d1080d53a293',
 '46c4630d-b0be-4d54-8b5d-74e6b48f3758',
 'd5b7119f-712c-4aac-b67f-519fa9ef2d1c',
 '9feaf202-1fed-455c-b8bf-ffcd4df56987',
 'c8fdb88d-1a13-495b-8e0f-9aaa05f8434c',
 'd5bf9eb6-1b22-4b17-9be1-48a7f28d8ce6',
 'dc824527-2673-43af-ba4a-9dbb94b05947',
 '1db156db-d713-4675-877e-818892ad89d4',
 '17cc5933-6bc3-4be6-a7cc-c0f79d413b91',
 '29acdb0f-aa61-4a2c-810c-a06b97907807',
 '9cec7f7f-7abb-

In [5]:
# Initate the retriever
retriever = vector_store.as_retriever(search_type = 'similarity',search_kwargs={'k':4})
# retriever.invoke("What is agent")

In [ ]:
# Working with tools
class Router(BaseModel):
    routing_to: Literal['websearch','doc_retriever'] = Field(description="Given a user question choose to route it to web search or a vectorstore.")
llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")
structured_llm_router = llm.with_structured_output(Router)
system = """You are an expert at routing a user question to either a vectorstore or a web search engine.
The vectorstore contains documents on the following topics:
- AI agents (including types of agent memory)
- Prompt engineering
- Adversarial attacks

If the user question is about any of these topics, route it to 'doc_retriever'.
Otherwise, route it to 'websearch'.

Respond with one of: 'websearch' or 'doc_retriever' only.
"""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
question_router = route_prompt | structured_llm_router
print(
    question_router.invoke(
        {"question": "who is Sharukh Khan?"}
    )
)
print(question_router.invoke({"question": "What are the types of agent memory?"}))


routing_to='websearch'
routing_to='doc_retriever'


In [7]:
# Grading the documennts
class DocumentGrader(BaseModel):
    grader_or_not: str = Field(description="Documents are relevant to the query or not")

llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
structured_llm_doc_grader = llm.with_structured_output(DocumentGrader)

grading_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "documents_for_grading {document} User question:{question}"),
    ]
)

document_grader = grading_prompt | structured_llm_doc_grader

question = "agent memory"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(document_grader.invoke({"question": question, "document": doc_txt}))

grader_or_not='yes'


In [8]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = ChatPromptTemplate.from_template("""
    You are an assistant. Use the following context to answer the question.

    Context:
    {context}

    Question:
    {question}

    Answer:""")

# LLM

llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

According to the provided context, the agent memory in a LLM-powered autonomous agent system consists of two types:

1. **Short-term memory**: This type of memory is utilized for in-context learning, which allows the agent to learn from its experiences and adapt to new situations.
2. **Long-term memory**: This type of memory enables the agent to retain and recall information over extended periods, often by leveraging an external vector store and fast retrieval. This allows the agent to retain knowledge and skills acquired over time.

Additionally, the context mentions that the agent's memory can be compared to the human brain's memory, which has different types such as sensory memory, iconic memory, echoic memory, and haptic memory.


In [ ]:
# Checking for hallucination
class HallucinationGrader(BaseModel):
    not_hallucinated: str = Field(description="Answer is grounded in facts. Must be 'yes' or 'no'.")
llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")
structured_llm_halluci_grader = llm.with_structured_output(HallucinationGrader)
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts.
Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Context:\n\n{document}\n\nResponse:\n\n{response}\n\nIs the response grounded in the context?"),
    ]
)
hallucination_grader = hallucination_prompt | structured_llm_halluci_grader
hallucination_grader.invoke({"document": docs, "response": generation})

HallucinationGrader(not_hallucinated='yes')

In [10]:
# Grading the answer

class AnswerGrader(BaseModel):
    graded_answer: str = Field(description="Answer is relevant and related to question. Must be 'yes' or 'no'.")

llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")
structured_llm_answer_grader = llm.with_structured_output(AnswerGrader)
system = """You are a grader assessing whether a response answers or addresses the user's question.
Give a binary score: 'yes' or 'no'. 'Yes' means the answer is relevant and resolves the question."""
answer_graded_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Question:\n\n{question}\n\nResponse:\n\n{response}\n\nDoes the response answer the question?"),
    ]
)
answer_grader = answer_graded_prompt | structured_llm_answer_grader
answer_grader.invoke({"question": question, "response": generation})

AnswerGrader(graded_answer='no')

In [11]:
# question_rewriter

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the input and You must only return the improved question — no commentary, no explanation with about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: {question} Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

"What are the key components of an agent's memory?"

In [12]:
class State(TypedDict):
    question: str
    response: str
    documents: List[str]
    generate_placeholder: str

In [ ]:
def generate(state:State):
    print("started generate")
    llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")
    
    # context = "\n\n".join(doc.page_content for doc in documents)

    # prompt = ChatPromptTemplate.from_template("""
    # You are an assistant. Use the following context to answer the question.

    # Context:
    # {context}

    # Question:
    # {question}

    # Answer:""")
    
    # llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")
    # rag_chain = prompt | llm | StrOutputParser()
    response = rag_chain.invoke({'question':state['question'],'context':state['documents']})
    print("The response of generate function is ", response)
    print("The document inside the generate",state['documents'])
    print("Ended Generate")
    return {"documents": state['documents'], "question": state['question'], "response": response}

def grade_documents(state:State):
    filtered_documents =[]
    for d in state['documents']:
        score = document_grader.invoke({"question": state['question'], "document": state['documents']})
        if score.grader_or_not == "yes":
            filtered_documents.append(d)
    return {"documents":filtered_documents}
    
def question_rewrite(state:State):
    print("Entered the question_rewriter")
    response = question_rewriter.invoke({"question":state['question']})
    print("The question has been rewrited successfully as ",response)
    return {'question':response}

def check_for_hallucination(state:State):
    print("Entered Hacculination")
    print("The document inside hallucination is", state['documents'])
    print("The response inside the hallucination",state['response'])
    score = hallucination_grader.invoke({"document":state['documents'], 'response':state['response']})
    print("The hallucination score is", score.not_hallucinated )
    
    if score.not_hallucinated == "yes":
        print("Started answer grader")
        response = answer_grader.invoke({'question':state['question'], 'response':state['response']})
        print("The response from answer grader", response.graded_answer)
        if response.graded_answer == "yes":
            print("Finished the process")
            return "finished"
        else:
            print("changing the query")
            return "transform_question"

    else:
        print("Again generation")
        return "generate"

def decision_to_generate(state:State):
    if state['documents']:
        return 'generate'
    else:
        return 'transform_question'

def web_search_tool(state:State):
    # api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
    # wikitool = WikipediaQueryRun(api_wrapper=api_wrapper)
    # result = wikitool.invoke({'query': state['question']})
    search = DuckDuckGoSearchRun()

    result = search.invoke({'query': state['question']})
    print("The wikitool result",result)
    documents = Document(page_content = result)
    return {'documents':documents,} # 'question':state['question']

def retrieve(state:State):
    print("Entered the retriever")
    print("The question in the retriever is ",state['question'])
    documents = retriever.invoke(state['question'])
    return {'documents':documents,} # 'question':state['question']

def router(state:State):
    result = question_router.invoke({'question':state['question']})
    print("Reached router")
    if result.routing_to == 'websearch':
        print("Inside websearch")
        return 'websearch'
    else:
        print("Inside websearch")
        return 'doc_retriever'


In [ ]:
graph_builder = StateGraph(State)
graph_builder.add_node('websearch',web_search_tool)
graph_builder.add_node('doc_retriever',retrieve)
graph_builder.add_node('generate',generate)
graph_builder.add_node('transform_question',question_rewrite)
graph_builder.add_node('grade_documents',grade_documents)

graph_builder.add_conditional_edges(START,router,{"websearch": "websearch", 'doc_retriever': 'doc_retriever'})
graph_builder.add_edge("transform_question", "doc_retriever")
graph_builder.add_conditional_edges('grade_documents',decision_to_generate,{'generate': 'generate','transform_question':'transform_question'})
graph_builder.add_edge("doc_retriever",'grade_documents')
graph_builder.add_edge( "websearch", 'generate')
graph_builder.add_conditional_edges('generate',check_for_hallucination,{"finished": END, 'generate': 'generate','transform_question':'transform_question'})
# Compile
app = graph_builder.compile()

In [ ]:
from pprint import pprint

# Run
# inputs = {
#     "question": "Avengers"
# }
inputs = {
    "question": "Agent Memory"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value['response'])

In [ ]:
print(inputs)
pprint(value['response'])